In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as sts
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [43]:
df = pd.read_csv("/content/Cleaned_df.txt")
df

,Index,BHK,Rent,Size,Floor,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,0,2,10000,1100,Ground out of 2,Super Area,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,1,2,20000,800,1 out of 3,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2,2,17000,1000,1 out of 3,Super Area,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,3,2,10000,800,1 out of 2,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,4,2,7500,850,1 out of 2,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...
4312,4741,2,15000,1000,3 out of 5,Carpet Area,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4313,4742,3,29000,2000,1 out of 4,Super Area,Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4314,4743,3,35000,1750,3 out of 5,Carpet Area,Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4315,4744,3,45000,1500,23 out of 34,Carpet Area,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [44]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4317 entries, 0 to 4316
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Index              4317 non-null   int64 
 1   BHK                4317 non-null   int64 
 2   Rent               4317 non-null   int64 
 3   Size               4317 non-null   int64 
 4   Floor              4317 non-null   object
 5   Area Type          4317 non-null   object
 6   City               4317 non-null   object
 7   Furnishing Status  4317 non-null   object
 8   Tenant Preferred   4317 non-null   object
 9   Bathroom           4317 non-null   int64 
 10  Point of Contact   4317 non-null   object
dtypes: int64(5), object(6)
memory usage: 371.1+ KB


,Index,BHK,Rent,Size,Bathroom
count,4317.000000,4317.000000,4317.000000,4317.000000,4317.000000
mean,2454.043317,1.974983,20410.776002,882.023164,1.824183
std,1375.280796,0.751782,15685.401394,497.800727,0.721951
min,0.000000,1.000000,1200.000000,10.000000,1.000000
25%,1339.000000,1.000000,9900.000000,550.000000,1.000000
50%,2505.000000,2.000000,15000.000000,800.000000,2.000000
75%,3637.000000,2.000000,25000.000000,1120.000000,2.000000
max,4745.000000,6.000000,79500.000000,4200.000000,7.000000


In [45]:
df.drop("Index",axis=1,inplace=True)
df["Point of Contact"] = df["Point of Contact"].replace({"Contact Owner":1, "Contact Agent":2, "Contact Builder":3})
df["City"] = df["City"].replace({"Mumbai":40, "Kolkata":70, "Hyderabad":50, "Delhi":11, "Chennai":60, "Bangalore":56})
df["Tenant Preferred"] = df["Tenant Preferred"].replace({"Bachelors/Family":2, "Bachelors":0, "Family":3})
df["Furnishing Status"] = df["Furnishing Status"].replace({"Unfurnished":0, "Semi-Furnished":1, "Furnished":2})
df["Area Type"] = df["Area Type"].replace({"Built Area":0, "Carpet Area":1, "Super Area":2})

floor = df['Floor'].str.split(' out of ').str[0]
df["floor"] = floor
df["floor"] = df["floor"].replace({"Ground":0, "Upper Basement":-1, "Lower Basement":-2}).astype(int)

total_floors = df['Floor'].str.split(' out of ').str[-1]
df["total_floors"] = total_floors
df["total_floors"] = df["total_floors"].replace({"Ground":0}).astype(int)

df.drop("Floor", axis=1, inplace=True)
df

,BHK,Rent,Size,Area Type,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,floor,total_floors
0,2,10000,1100,2,70,0,2,2,1,0,2
1,2,20000,800,2,70,1,2,1,1,1,3
2,2,17000,1000,2,70,1,2,1,1,1,3
3,2,10000,800,2,70,0,2,1,1,1,2
4,2,7500,850,1,70,0,0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
4312,2,15000,1000,1,50,1,2,2,1,3,5
4313,3,29000,2000,2,50,1,2,3,1,1,4
4314,3,35000,1750,1,50,1,2,3,2,3,5
4315,3,45000,1500,1,50,1,3,2,2,23,34


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4317 entries, 0 to 4316
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   BHK                4317 non-null   int64
 1   Rent               4317 non-null   int64
 2   Size               4317 non-null   int64
 3   Area Type          4317 non-null   int64
 4   City               4317 non-null   int64
 5   Furnishing Status  4317 non-null   int64
 6   Tenant Preferred   4317 non-null   int64
 7   Bathroom           4317 non-null   int64
 8   Point of Contact   4317 non-null   int64
 9   floor              4317 non-null   int64
 10  total_floors       4317 non-null   int64
dtypes: int64(11)
memory usage: 371.1 KB


In [47]:
x = df.drop(["Rent"], axis=1)
y = df["Rent"]

x_train, x_test, y_train, y_test = tts(x, y, train_size=0.8, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3453, 10) (864, 10) (3453,) (864,)


In [48]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [49]:
y_pred = model.predict(x_test)

In [52]:
bhk = int(input("Enter the house BHK: "))
size = int(input("Enter the house size: "))
city = int(input("The codes for different cities are: {Mumbai:40, Kolkata:70, Hyderabad:50, Delhi:11, Chennai:60, Bangalore:56}. Enter the code for the city: "))
floor = int(input("The codes for different floors are {Lower Basement:-2, Upper Basement:-1, Ground:0,......and so on.}. Enter the house floor: "))
total_floors = int(input("If there are only one floor, which is ground, enter 0, other wise enter total number of floors, counting starting from 0. Enter the house total floors: "))
contact = int(input("For point of contact the codes are: {Contact Owner:1, Contact Agent:2, Contact Builder:3}. Enter the point of contact : "))
tenant_preferred = int(input("For tenant preferred the codes are: {Bachelors/Family:2, Bachelors:0, Family:3}. Enter the tenant preferred : "))
furnishing_status = int(input("For furnishing status the codes are: {Unfurnished:0, Semi-Furnished:1, Furnished:2}. Enter the furnishing status : "))
area_type = int(input("For area type the codes are: {Built Area:0, Carpet Area:1, Super Area:2}. Enter the area type : "))
bathroom = int(input("Enter the number of bathrooms: "))

data = [[bhk, size, area_type, city, furnishing_status, tenant_preferred, bathroom, contact, floor, total_floors]]
column = ['BHK', 'Size', 'Area Type', 'City', 'Furnishing Status', 'Tenant Preferred', 'Bathroom', 'Point of Contact', 'floor', 'total_floors']
inp = pd.DataFrame(data, columns=column)

city_dict = {40:"Mumbai", 70:"Kolkata", 50:"Hyderabad", 11:"Delhi", 60:"Chennai", 56:"Bangalore"}
city = city_dict[city]
house_rent = model.predict(inp).astype('int32')
print("The approximate rent to pay in the city", city, "for", bhk, "bhk is rupees",house_rent[0],'.')

Enter the house BHK: 2
Enter the house size: 400
The codes for different cities are: {Mumbai:40, Kolkata:70, Hyderabad:50, Delhi:11, Chennai:60, Bangalore:56}. Enter the code for the city: 11
The codes for different floors are {Lower Basement:-2, Upper Basement:-1, Ground:0,......and so on.}. Enter the house floor: 4
If there are only one floor, which is ground, enter 0, other wise enter total number of floors, counting starting from 0. Enter the house total floors: 4
For point of contact the codes are: {Contact Owner:1, Contact Agent:2, Contact Builder:3}. Enter the point of contact : 1
For tenant preferred the codes are: {Bachelors/Family:2, Bachelors:0, Family:3}. Enter the tenant preferred : 0
For furnishing status the codes are: {Unfurnished:0, Semi-Furnished:1, Furnished:2}. Enter the furnishing status : 0
For area type the codes are: {Built Area:0, Carpet Area:1, Super Area:2}. Enter the area type : 0
Enter the number of bathrooms: 1
The approximate rent to pay in the city Delhi